In [1]:
import os 
import pandas as pd
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms

In [ ]:
# Download with: python -m spacy download en
spacy_eng = spacy.load("en")

class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

In [ ]:
df = pd.read_csv('captions.txt', delimiter = '	', names = ['image', 'caption'])
vocab = Vocabulary(freq_threshold=5)
vocab.build_vocabulary(df['caption'].tolist())

In [ ]:
numericalized_caption = [vocab.stoi["<SOS>"]]
numericalized_caption += vocab.numericalize(df['caption'][0])
numericalized_caption.append(vocab.stoi["<EOS>"])

In [ ]:
numericalized_caption, df['caption'][0]

In [13]:
dfx = pd.read_csv('captions.txt', delimiter = '	', names = ['image', 'caption'])

In [14]:
dff = pd.read_csv('image_names.txt', names = ['image'])

In [15]:
images = dff['image'].tolist()

In [16]:
imgs = dfx["image"].tolist()
dfx = dfx.set_index('image')
for im in imgs:
    if im[:-2] not in images:
        dfx = dfx.drop(im, axis=0)

In [18]:
dfx.to_csv('caption.csv')